In [ ]:
!python3 -m pip install PyMySQL
!python3 -m pip install SQLAlchemy
!python3 -m pip install google-cloud-storage
!python3 -m pip install --upgrade --quiet scikit-sound
!python3 -m pip install --upgrade --quiet pygame
!sudo apt-get -y install ffmpeg
!sudo apt-get -y install python3-pymysql

In [2]:
import sqlalchemy
import numpy as np

from google.cloud import storage
from numpy.fft import fft, ifft
from sksound.sounds import Sound
from scipy.io import wavfile

from tensorflow.keras.models import load_model

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
highpass=25

class StreamSpectrogram:
    
    def __init__(self, filename, win=128):
        sound = Sound(filename)        
        self.batch = sound.rate * 60
        self.data  = sound.data
        self.fs    = sound.rate
        if len(self.data.shape) > 1:
            self.data = self.data[:, 0]    
        self.win = win
        
    def __iter__(self):
        self.i = 0
        return self

    def __next__(self):
        if (self.i + 1) * self.batch < len(self.data): 
            start = self.i       * self.batch
            stop  = (self.i + 1) * self.batch
            raw   = self.data[start:stop]
            spec  = fwd_spectrogram(raw, win=512 + 2 * highpass)[:, 0:256] 
            t,d   = spec.shape
            current = []        
            for i in range(self.win, t, self.win // 2):
                x      = np.reshape(spec[i - self.win:i], (self.win, d, 1))
                mu     = np.mean(x)
                std    = np.std(x) + 1.0
                window = (x - mu) / std
                current.append(window)
            self.i += 1
            return np.stack(current)
        else:
            raise StopIteration
     
    def snippet(self, start, stop):
        w = 512 + 2 * highpass
        if start - w > 0:
            return self.data[start - w:stop]
        else:
            return None
    
def fwd_spectrogram(audio, win=512, step=64):
    '''
    Compute the spectrogram of audio data

    audio: one channel audio
    win: window size for dft sliding window
    step: step size for dft sliding windo
    '''
    spectrogram = []
    hanning = np.hanning(win)
    for i in range(win, len(audio), step):
        start = win // 2
        dft = np.abs(fft(audio[i - win: i] * hanning))[start:win]
        spectrogram.append(dft)
    return np.array(spectrogram)

In [4]:
# PASSWORDS AND STUFF HERE

In [5]:
settings = {
   'user': db_user,
   'pass': db_password,
   'host': host,
     'db': db_name
}
url = 'mysql+pymysql://{user}:{pass}@{host}/{db}'.format(**settings)  # 5432 is the default port
db = sqlalchemy.create_engine(url)

In [6]:
def run_query(query):
    with db.connect() as conn:
        rows = []
        for row in conn.execute(query).fetchall():
            rows.append(dict(row.items()))
        return rows

In [7]:
files = run_query("""
    SELECT 
        x.encoding, y.year, x.filename 
    FROM 
        wdp_ds.audio x 
    JOIN wdp_ds.encoding y ON x.encoding = y.encoding;
""")

In [8]:
paths = ["audio_files/{}/{}".format(file['year'], file['filename']) for file in files]

In [11]:
max_id = run_query("""
    SELECT 
        max(id)
    FROM 
        wdp_ds.not_silent
    """)
if max_id[0]['max(id)'] is None:
    max_id = 0
else:
    max_id = max_id[0]['max(id)']
print(max_id)

0


In [ ]:
import sys
sys.path.append("../ml") 

models = '../models/lstm_v4/v4.2/'

from train_silence_detector import detector
from train_lstm_auto_encoder_variational import VAE

vae = VAE()
vae.auto_encoder((128, 256, 1), 128, 256 * 128, 128)
encoder = vae.encoder 
noise_classifier = detector(128, encoder)
encoder.load_weights('{}encoder.h5'.format(models))
noise_classifier.load_weights('{}sil.h5'.format(models))
encoder.summary()


#noise_classifier = load_model('../models/lstm_v4/v4.2/sil.h5')
client = storage.Client.from_service_account_json('../secret.json')
bucket = client.get_bucket('wdp-data')

In [ ]:
id = max_id + 1
skip = 0
c = 0
for path, file_dict in zip(paths, files):
    if c > skip:
        print(path, file_dict, c)
        with open("/tmp/audio.m4a", "wb") as file_obj:
            blob = bucket.blob(path)
            blob.download_to_file(file_obj)
        stream = StreamSpectrogram("/tmp/audio.m4a")
        not_noise = []
        for x in stream:
            y = noise_classifier.predict(x).flatten()
            not_noise.extend([int(np.round(sample)) == 0 for sample in y])

        regions = []
        for i in range(0, len(not_noise)):
            if not_noise[i]:
                #win: 32, step: 256
                #start = i * 16 * 256
                #stop  = (i + 1) * 16 * 256 
                # win: 128, step: 64
                start = i * 64 * 64
                stop  = (i + 1) * 64 * 64 

                if len(regions) > 0: 
                    last  = regions[-1]
                    if start - last[1] < 48000 * 0.1:
                        start       = regions[-1][0]
                        regions[-1] = (start, stop)
                    else:
                        regions.append((start, stop))
                else:
                    regions.append((start, stop))
        regions = [(start, stop) for start, stop in regions if stop - start > (64 * 64)]
        if len(regions) > 0:
            #for start, stop in regions:
            #    audio = stream.snippet(start, stop)
            #   if audio is not None:
            #       wavfile.write('../data/silence/{}_{}_{}.wav'.format(file_dict['encoding'], start,stop), stream.fs, audio)

            with db.connect() as conn:
                for start, stop in regions:                
                    conn.execute("INSERT INTO wdp_ds.not_silent VALUES ({}, {}, '{}', {}, {})".format(id, file_dict['encoding'], file_dict['filename'], start, stop))  
                    id += 1
        print(file_dict['encoding'], ": ", regions)
    c += 1